In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim
from torch.autograd import Variable
from scipy.stats import spearmanr
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1

In [27]:

# Define the SCAD penalty and its derivative
def scad_penalty(beta, lambda_val, a_val):
    abs_beta = torch.abs(beta)
    is_linear = abs_beta <= lambda_val
    is_quadratic = (lambda_val < abs_beta) & (abs_beta <= a_val * lambda_val)
    is_constant = abs_beta > a_val * lambda_val

    linear_part = lambda_val * abs_beta * is_linear
    quadratic_part = (a_val * lambda_val * abs_beta - beta**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part

def scad_derivative(beta, lambda_val, a_val):
    indicator = beta.abs() <= lambda_val
    value = lambda_val * (indicator + (a_val * lambda_val - beta.abs()).clamp(min=0) * (1 - indicator) / ((a_val - 1) * lambda_val))
    return torch.sign(beta) * value

# PyTorch model with SCAD regularization
class SCADLinearModel(nn.Module):
    def __init__(self, n_features, a=3.7, lambda_=1.0, device='cpu'):
        super(SCADLinearModel, self).__init__()
        self.linear = torch.nn.Linear(n_features, 1, bias=False).to(device)
        self.a = a
        self.lambda_ = lambda_
        self.device = device

    def forward(self, x):
        return self.linear(x)

    def regularization(self, beta):
        return scad_penalty(beta, self.lambda_, self.a).sum()

    def fit(self, X, y, lr=1e-6, epochs=100):
        # Define the optimizer with a lower learning rate
        optimizer = torch.optim.SGD(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.forward(X)
            loss = torch.nn.functional.mse_loss(outputs, y.reshape(-1, 1)) + self.regularization(self.linear.weight)
            loss.backward()
            optimizer.step()
            if epoch % 10 == 0:
                print(f'Epoch {epoch}: Loss {loss.item()}')

    # Add a predict method
    def predict(self, X):
        # Ensure model is in evaluation mode
        self.eval()
        with torch.no_grad():
            predictions = self.forward(X)
        return predictions

    # Implement the get_coefficients method
    def get_coefficients(self):
        return self.linear.weight.data

# Example usage
# Note: X and y should be loaded and preprocessed as PyTorch tensors before calling fit.
# model = SCADLinearModel(n_features=X.shape[1], device='cuda' if torch.cuda.is_available() else 'cpu')
# model.fit(X_train, y_train)


In [10]:
df = pd.read_csv('drive/MyDrive/advData/housing.csv')
features = ['crime','rooms','residential','industrial','nox','older','distance','highway','tax','ptratio','lstat']
X = np.array(df[features])
y = np.array(df['cmedv']).reshape(-1,1)

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float64)
y_tensor = torch.tensor(y, dtype=torch.float64)

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)

# Initialize and fit the SCAD model
n_features = X_tensor.shape[1]


# Initialize and fit the SCAD model
model = SCADLinearModel(n_features=X.shape[1], device=device)
model.double()  # Convert all model parameters to double precision
model.fit(X_tensor, y_tensor)




Epoch 0: Loss 26868.32276918386
Epoch 10: Loss 199.42032117161745
Epoch 20: Loss 195.8466648792458
Epoch 30: Loss 193.2703528153648
Epoch 40: Loss 190.75696348161915
Epoch 50: Loss 188.30486561467242
Epoch 60: Loss 185.91250249851646
Epoch 70: Loss 183.57835819395535
Epoch 80: Loss 181.30095643392852
Epoch 90: Loss 179.07885955084973


In [11]:
# Retrieve the learned weights
learned_weights = model.linear.weight.detach().cpu().numpy().flatten()

# Analyze feature importance based on the magnitude of weights
feature_importance = np.abs(learned_weights)

# Select features with non-zero or significant weights
selected_features_indices = np.where(feature_importance > 0)[0]
selected_features = [features[i] for i in selected_features_indices]

# Print the selected features
print("Selected features based on SCAD feature importance:")
for feature in selected_features:
    print(feature)

Selected features based on SCAD feature importance:
crime
rooms
residential
industrial
nox
older
distance
highway
tax
ptratio
lstat


# This suggests that all features have some degree of importance in predicting the target variable.

# 2

In [14]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        loss = mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * l2_reg
        )

        return loss

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight


In [23]:
# we can call this version PED_Adam because we use the adaptive momentum gradient descent for optimization
class sqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        """
        Initialize the  regression model.


        """
        super(sqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha


        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1).float()

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = (len(y_true)*mse_loss)**(1/2) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

In [28]:


# Assuming definitions of ElasticNet, sqrtLasso, and SCADLinearModel are as provided earlier

def generate_dataset(n_samples, n_features, correlation, betastar, noise_std=0.1):
    cov_matrix = correlation * np.ones((n_features, n_features)) + \
                 (1 - correlation) * np.eye(n_features)
    X = np.random.multivariate_normal(np.zeros(n_features), cov_matrix, n_samples)
    noise = np.random.normal(0, noise_std, n_samples)
    y = X.dot(betastar) + noise
    return X, y

def train_and_evaluate(model, X_train, y_train, X_test, y_test, betastar):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Ensure y_train is 2D for compatibility
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Ensure y_test is 2D for compatibility

    if isinstance(model, ElasticNet):
        model.fit(X_train_tensor, y_train_tensor, num_epochs=100)  # Use num_epochs for ElasticNet
    elif isinstance(model, sqrtLasso):
        model.fit(X_train_tensor, y_train_tensor, num_epochs=200)  # Adjust according to sqrtLasso definition
    elif isinstance(model, SCADLinearModel):
        model.fit(X_train_tensor, y_train_tensor, epochs=100)  # Adjust according to SCADLinearModel definition

    y_pred = model.predict(X_test_tensor)
    mse = torch.nn.functional.mse_loss(y_pred, y_test_tensor).item()

    # Convert betastar to a PyTorch tensor for compatibility in subtraction
    betastar_tensor = torch.tensor(betastar, dtype=torch.float32)
    coefficient_recovery_error = torch.sum((betastar_tensor - model.get_coefficients().detach().flatten()) ** 2).item()

    return mse, coefficient_recovery_error




n_samples = 100
n_features = 20
n_datasets = 500
correlation = 0.9
betastar = np.zeros(n_features)
betastar[:5] = np.random.uniform(1, 3, 5)  # Assuming the first 5 features are relevant

results = {
    'ElasticNet': {'mse': [], 'recovery_error': []},
    'SqrtLasso': {'mse': [], 'recovery_error': []},
    'SCAD': {'mse': [], 'recovery_error': []}
}

for _ in range(n_datasets):
    X, y = generate_dataset(n_samples, n_features, correlation, betastar)
    # Splitting dataset into 80% training and 20% testing
    split_index = int(n_samples * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # ElasticNet Model
    en_model = ElasticNet(n_features)
    mse_en, reco_err_en = train_and_evaluate(en_model, X_train, y_train, X_test, y_test, betastar)
    results['ElasticNet']['mse'].append(mse_en)
    results['ElasticNet']['recovery_error'].append(reco_err_en)

    # SqrtLasso Model
    sqrt_lasso_model = sqrtLasso(n_features)
    mse_sqrt_lasso, reco_err_sqrt_lasso = train_and_evaluate(sqrt_lasso_model, X_train, y_train, X_test, y_test, betastar)
    results['SqrtLasso']['mse'].append(mse_sqrt_lasso)
    results['SqrtLasso']['recovery_error'].append(reco_err_sqrt_lasso)

    # SCAD Model
    scad_model = SCADLinearModel(n_features)
    mse_scad, reco_err_scad = train_and_evaluate(scad_model, X_train, y_train, X_test, y_test, betastar)
    results['SCAD']['mse'].append(mse_scad)
    results['SCAD']['recovery_error'].append(reco_err_scad)

# Average MSE and coefficient recovery error for each model
average_results = {model: {metric: np.mean(values) for metric, values in metrics.items()} for model, metrics in results.items()}
print(average_results)


Streaming output truncated to the last 5000 lines.
Epoch 50: Loss 129.95706176757812
Epoch 60: Loss 129.86134338378906
Epoch 70: Loss 129.7657012939453
Epoch 80: Loss 129.670166015625
Epoch 90: Loss 129.5746612548828
Epoch [10/100], Loss: 9.19581413269043
Epoch [20/100], Loss: 9.100560188293457
Epoch [30/100], Loss: 9.019798278808594
Epoch [40/100], Loss: 8.945869445800781
Epoch [50/100], Loss: 8.878151893615723
Epoch [60/100], Loss: 8.816079139709473
Epoch [70/100], Loss: 8.759135246276855
Epoch [80/100], Loss: 8.706853866577148
Epoch [90/100], Loss: 8.658811569213867
Epoch [100/100], Loss: 8.614625930786133
Epoch [100/200], Loss: 13.899430137872695
Epoch [200/200], Loss: 12.086057376861572
Epoch 0: Loss 139.0550537109375
Epoch 10: Loss 138.94439697265625
Epoch 20: Loss 138.83384704589844
Epoch 30: Loss 138.723388671875
Epoch 40: Loss 138.61302185058594
Epoch 50: Loss 138.5027618408203
Epoch 60: Loss 138.392578125
Epoch 70: Loss 138.282470703125
Epoch 80: Loss 138.17247009277344
Epoch

# ElasticNet produces the best approximation of an ideal solution

# 3

https://brukeamare.github.io/AdvancedAML/